This file takes as input the detected communities from the Louvain algorithm and outputs the groups of communities

## Spark setup & libraries

In [ ]:
# spark setup
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as F
from pyspark.ml.feature import MinHashLSH, VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col, when, explode, lit, array_contains
# from itertools import groupby
from pyspark.sql import Window
from pyspark.sql.functions import row_number, ceil, col, udf
import math

import networkx as nx
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster
from collections import defaultdict

# libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

# show all rows with df.head
pd.options.display.max_columns = None
# remove warnings
import warnings
warnings.filterwarnings('ignore')

## Create spark session + load data

In [ ]:
spark = SparkSession.builder \
  .appName("DIS_project_5") \
  .master("local[*]") \
  .config("spark.driver.memory", "10G") \
  .config("spa\rk.driver.maxResultSize", "40g") \
  .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
  .getOrCreate()
spark
sc = spark.sparkContext

In [ ]:
json_file_path = "../Community Detection/20Knode_20edge.json"
with open(json_file_path, 'r') as f:
    community_data = json.load(f)

community_rdd = sc.parallelize(community_data)
print(community_rdd.take(3))
print(community_rdd.count())
print(type(community_rdd))

## Group communities

In [ ]:
# turn rdd into dataframe and add columns for count of nodes and edges
df_community = spark.createDataFrame(community_rdd)
df_community.show()
# Add columns for node size and edge size
df_with_sizes = df_community \
    .withColumn("node_size", F.size("nodes")) \
    .withColumn("edge_size", F.size("edges")) \

df_with_sizes.show()

# group communities by number of nodes
# Group by node size
node_size_groups = df_with_sizes.groupBy("node_size").count()
node_size_groups.show()


# group communities by number of edges
# Group by edge size
edge_size_groups = df_with_sizes.groupBy("edge_size").count()
edge_size_groups.show()

### Make N groups by size

In [ ]:
# group communities by number of nodes and edges
def group_by_size(df, num_groups, column_name):
  # sort the data by the column name
  sorted_df = df.sort(column_name)

  # calculate the size of each group
  group_size = math.ceil(df.count() / num_groups)
  
  # add a row number column to the dataframe
  df_with_row_number = df.withColumn("row_number", row_number().over(Window.orderBy(column_name)))

  # use row number to create groups
  df_groups = df_with_row_number.withColumn("group", ceil(col("row_number") / group_size)).drop("row_number")

  return df_groups


result = group_by_size(df_with_sizes, 3, 'node_size')
result.show(55)

### Apply Unique Pairing

In [ ]:
# make unique pairs of communities in same group
def create_pairs_grouped_communities(df_groups):
  df_pairs = (
    df_groups.alias("df1")
    .join(df_groups.alias("df2"), (F.col("df1.group") == F.col("df2.group")) & (F.col("df1.community_id") < F.col("df2.community_id")))
    # .withColumn("pairs", (F.col("df1.community_id") + F.lit(", ") + F.col("df2.community_id")))
    .select(
      F.array(F.col("df1.community_id"), F.col("df2.community_id")).alias("pairs"),
      F.col("df1.group").alias("group"),
      F.col("df1.community_id").alias("community_id_1"),
      F.col("df2.community_id").alias("community_id_2"),
      F.col("df1.nodes").alias("nodes_1"),
      F.col("df2.nodes").alias("nodes_2"),
      F.col("df1.edges").alias("edges_1"),
      F.col("df2.edges").alias("edges_2"),
    )
  )

  return df_pairs

result_group_communities = create_pairs_grouped_communities(result)
result_group_communities.show()

In [ ]:
# Turn rdd of communities into dataframe of pairs of communities of similiar node or edge size
def rdd_community_to_dataframe_paired_community_by_column(rdd_communities, number_groups_of_communities_wanted, column_name):
  # turn rdd into dataframe and add columns for count of nodes and edges
  df_community = spark.createDataFrame(rdd_communities)
  df_with_sizes = df_community \
    .withColumn("node_size", F.size("nodes")) \
    .withColumn("edge_size", F.size("edges")) \

  # group communities by number of nodes and edges
  df_communities_groups = group_by_size(df_with_sizes, number_groups_of_communities_wanted, column_name)

  # make unique pairs of communities in same group
  result_grouped_communities = create_pairs_grouped_communities(df_communities_groups)

  # from this dataframe you can see which communities are grouped together, so all the pairs as a seperate column

  return result_grouped_communities


In [ ]:
#Creating Dataframe of grouped communities
grouped_communities=rdd_community_to_dataframe_paired_community_by_column(community_rdd, 3, 'node_size')

#Converting it to RDD
pairs_rdd = grouped_communities.rdd
type(pairs_rdd)

### Define Temporal NetSimile

In [ ]:
###TemporalNetSimile defintion
"""
Temporalnetsimile.py
--------------

Graph distance based on:
Berlingerio, M., Koutra, D., Eliassi-Rad, T. & Faloutsos, C. NetSimile: A Scalable Approach to Size-Independent Network Similarity. arXiv (2012)

Extended upon netsimile.py, acreditted to:

author: Alex Gates
email: ajgates42@gmail.com (optional)
Submitted as part of the 2019 NetSI Collabathon.
https://netrd.readthedocs.io/en/latest/_modules/netrd/distance/netsimile.html

We wish to note that extensions upon the initial netsimile code is preceded by a ###comment

"""

import networkx as nx
import numpy as np
from scipy.spatial.distance import canberra
from scipy.stats import skew, kurtosis

from netrd.distance.base import BaseDistance
from netrd.utilities import undirected, unweighted

###Needed to convert time later
from datetime import datetime


class TemporalNetSimile(BaseDistance):
    """Compares node signature distributions."""


    @undirected
    @unweighted
    def dist(self, G1, G2):
        """A scalable approach to network similarity.

        A network similarity measure based on node signature distributions.
        
        The results dictionary includes the underlying feature matrices in
        `'feature_matrices'` and the underlying signature vectors in
        `'signature_vectors'`.

        Parameters
        ----------

        G1, G2 (nx.Graph)
            two undirected networkx graphs to be compared.

        Returns
        -------

        dist (float)
            the distance between `G1` and `G2`.

        References
        ----------

        .. [1] Michele Berlingerio, Danai Koutra, Tina Eliassi-Rad,
               Christos Faloutsos: NetSimile: A Scalable Approach to
               Size-Independent Network Similarity. CoRR abs/1209.2684
               (2012)

        """

        # find the graph node feature matrices
        G1_node_features = feature_extraction(G1)
        G2_node_features = feature_extraction(G2)

        # get the graph signature vectors
        G1_signature = graph_signature(G1_node_features)
        G2_signature = graph_signature(G2_node_features)

        # the final distance is the absolute canberra distance
        dist = abs(canberra(G1_signature, G2_signature))

        ###Remove for quicker comp.
        ###self.results['feature_matrices'] = G1_node_features, G2_node_features
        ###self.results['signature_vectors'] = G1_signature, G2_signature
        
        self.results['dist'] = dist

        return dist



def feature_extraction(G):
    """Node feature extraction.

    Parameters
    ----------

    G (nx.Graph): a networkx graph.

    Returns
    -------

    node_features (float): the Nx7 matrix of node features."""

    # necessary data structures
    ###FEATURE NUMBER
    node_features = np.zeros(shape=(G.number_of_nodes(), 10)) 
    node_list = sorted(G.nodes())
    node_degree_dict = dict(G.degree())
    node_clustering_dict = dict(nx.clustering(G))
    egonets = {n: nx.ego_graph(G, n) for n in node_list}

    # node degrees
    degs = [node_degree_dict[n] for n in node_list]

    # clustering coefficient
    clusts = [node_clustering_dict[n] for n in node_list]

    # average degree of neighborhood
    neighbor_degs = [
        np.mean([node_degree_dict[m] for m in egonets[n].nodes if m != n])
        if node_degree_dict[n] > 0
        else 0
        for n in node_list
    ]

    # average clustering coefficient of neighborhood
    neighbor_clusts = [
        np.mean([node_clustering_dict[m] for m in egonets[n].nodes if m != n])
        if node_degree_dict[n] > 0
        else 0
        for n in node_list
    ]

    # number of edges in the neighborhood
    neighbor_edges = [
        egonets[n].number_of_edges() if node_degree_dict[n] > 0 else 0
        for n in node_list
    ]

    # number of outgoing edges from the neighborhood
    # the sum of neighborhood degrees = 2*(internal edges) + external edges
    # node_features[:,5] = node_features[:,0] * node_features[:,2] - 2*node_features[:,4]
    neighbor_outgoing_edges = [
        len(
            [
                edge
                for edge in set.union(*[set(G.edges(j)) for j in egonets[i].nodes])
                if not egonets[i].has_edge(*edge)
            ]
        )
        for i in node_list
    ]

    # number of neighbors of neighbors (not in neighborhood)
    neighbors_of_neighbors = [
        len(
            set([p for m in G.neighbors(n) for p in G.neighbors(m)])
            - set(G.neighbors(n))
            - set([n])
        )
        if node_degree_dict[n] > 0
        else 0
        for n in node_list
    ]

    ###Temporal features on the edges

    ###start time, average start time of the edges of a single node
    ###Loop over all nodes, look at the starttime data from the edges, convert the time to time format, calculate mean start time for each node
    startTime = []
    for n in node_list:
        start_times = []
        for u, v, data in G.edges(n, data=True):
            # Convert begintijd to a datetime object
            start_time_str = str(data['begintijd'])
            start_time = datetime.strptime(start_time_str, '%Y%m%d%H%M%S')
            start_times.append(start_time.timestamp())  # Store as timestamp (seconds since epoch)

        startTime.append(np.mean(start_times) if start_times else 0)

    ###end time, average end time of the edges of a single node
    ###Loop over all nodes, look at the endtime data from the edges, convert the time to time format, calculate mean end time for each node
    endTime = []
    for n in node_list:
        end_times = []
        for u, v, data in G.edges(n, data=True):
            # Convert begintijd to a datetime object
            end_time_str = str(data['eindtijd'])
            end_time = datetime.strptime(end_time_str, '%Y%m%d%H%M%S')
            end_times.append(end_time.timestamp())  # Store as timestamp (seconds since epoch)

        endTime.append(np.mean(end_times) if end_times else 0)
    
    ###duration, average duration of the edges of a single node
    duration = []
    for n in node_list:
        duration.append(endTime[node_list.index(n)] - startTime[node_list.index(n)])
    ### end change

    # assembling the features
    node_features[:, 0] = degs
    node_features[:, 1] = clusts
    node_features[:, 2] = neighbor_degs
    node_features[:, 3] = neighbor_clusts
    node_features[:, 4] = neighbor_edges
    node_features[:, 5] = neighbor_outgoing_edges
    node_features[:, 6] = neighbors_of_neighbors

    ###Time features
    node_features[:, 7] = startTime
    node_features[:, 8] = endTime
    node_features[:, 9] = duration

    return np.nan_to_num(node_features)


def graph_signature(node_features):
    ###FEATURE NUMBER
    signature_vec = np.zeros(10 * 5)

    # for each of the features
    ### FEATURE NUMBER
    for k in range(10):
        # find the mean
        signature_vec[k * 5] = node_features[:, k].mean()
        # find the median
        signature_vec[k * 5 + 1] = np.median(node_features[:, k])
        # find the std
        signature_vec[k * 5 + 2] = node_features[:, k].std()
        # find the skew
        signature_vec[k * 5 + 3] = skew(node_features[:, k])
        # find the kurtosis
        signature_vec[k * 5 + 4] = kurtosis(node_features[:, k])

    return signature_vec


"""
# sample usage
>>>G1 = nx.karate_club_graph()
>>>G2 = nx.krackhardt_kite_graph()

>>>test = TemporalNetSimile()
>>>print(test.dist(G1, G2))
11.45
"""

### Convert data to graph, run Temporal NetSimile

In [ ]:
# Define function that takes data from two communities and output a Distance
def Compare(Community1, Community2):
    #Convert into networkX graph to use as input for TemportalNetSimile
    #Load Graphs
    Graph1 = nx.Graph()
    Graph1.add_nodes_from(Community1["nodes"])

    # Add edges with timestamps as edge attributes
    for edge in Community1["edges"]:
        Graph1.add_edge(edge["node1"], edge["node2"], begintijd=edge["begintijd"], eindtijd=edge["eindtijd"])

    Graph2=nx.Graph()
    Graph2.add_nodes_from(Community2["nodes"])
    for edge in Community2["edges"]:
        Graph2.add_edge(edge["node1"], edge["node2"], begintijd=edge["begintijd"], eindtijd=edge["eindtijd"])

    testTime=TemporalNetSimile()
    Distance = testTime.dist(Graph1, Graph2)
    return(Distance)

#print(Compare(Community1,Community2))

### Compute similarity for every pair

In [ ]:
# Step 2: Define a function to compute similarity
def compute_similarity(row):
    # Extract community data from the row
    community1 = {
        'community_id': row['community_id_1'],
        'nodes': row['nodes_1'],
        'edges': row['edges_1']
    }
    community2 = {
        'community_id': row['community_id_2'],
        'nodes': row['nodes_2'],
        'edges': row['edges_2']
    }
    group = row['group']

    # Compute the distance using the Compare function
    distance = Compare(community1, community2)

    # Return a tuple: (group, community_id1, community_id2, distance)
    return (group, community1['community_id'], community2['community_id'], distance)


In [ ]:
# Step 3: Apply the function to each row in the RDD
similarities_rdd = pairs_rdd.map(compute_similarity)

# Step 4: Collect the results
similarities = similarities_rdd.collect()


### Organize by group

In [ ]:
# Organize similarities by group
group_similarities = defaultdict(list)
for group, community_id1, community_id2, distance in similarities:
    group_similarities[group].append((community_id1, community_id2, distance))

print(group_similarities)

## Assign communities to groups based on a threshold using average linking

In [ ]:
import numpy as np
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, fcluster

for group, group_sims in group_similarities.items():
    # Get the set of unique community IDs in the group
    community_ids = set()
    for community_id1, community_id2, _ in group_sims:
        community_ids.add(community_id1)
        community_ids.add(community_id2)
    community_ids = sorted(community_ids)
    id_to_index = {community_id: idx for idx, community_id in enumerate(community_ids)}
    N = len(community_ids)
    
    # Initialize a full distance matrix with zeros or a large number for missing values
    distance_matrix = np.full((N, N), np.inf)
    np.fill_diagonal(distance_matrix, 0)  # Distance to self is zero

    # Fill in the distances
    for community_id1, community_id2, distance in group_sims:
        i = id_to_index[community_id1]
        j = id_to_index[community_id2]
        distance_matrix[i, j] = distance
        distance_matrix[j, i] = distance  # Symmetric matrix
    
    # Convert to condensed distance matrix required by linkage function
    condensed_dists = squareform(distance_matrix)

    # Apply hierarchical clustering
    Z = linkage(condensed_dists, method='average')  # You can try other methods like 'single', 'complete', etc.

    # Choose a threshold or number of clusters
    
    distances = [distance for _, _, distance in group_sims]
    mean_distance = np.mean(distances)
    median_distance = np.median(distances)
    std_distance = np.std(distances)
    print(f"Group {group} - Mean: {mean_distance}, Median: {median_distance}, Std: {std_distance}")

    threshold = mean_distance - std_distance
    print(f"Group {group} - Threshold: {threshold}")
    # Adjust based on your analysis for this group
    # Option 2: Specify the number of clusters
    # num_clusters = 3  # Adjust based on desired number of clusters

    # Get cluster labels
    cluster_labels = fcluster(Z, t=threshold, criterion='distance')
    # cluster_labels = fcluster(Z, t=num_clusters, criterion='maxclust')

    # Group communities by cluster labels
    clusters = defaultdict(list)
    for community_id, cluster_label in zip(community_ids, cluster_labels):
        clusters[cluster_label].append(community_id)
    
    # Output the clusters for this group
    print(f"\nGroup {group} Clustering Results:")
    for cluster_label, community_list in clusters.items():
        print(f"  Cluster {cluster_label}: {sorted(community_list)}")


### Visualize the distances within groups

In [ ]:
import matplotlib.pyplot as plt

for group, group_sims in group_similarities.items():
    # Extract distances
    distances = [distance for _, _, distance in group_sims]
    
    # Plot histogram of distances for the group
    plt.figure()
    plt.hist(distances, bins=20)
    plt.title(f'Group {group} Distance Distribution')
    plt.xlabel('Distance')
    plt.ylabel('Frequency')
    plt.show()
    
    # Calculate statistical measures
    mean_distance = np.mean(distances)
    median_distance = np.median(distances)
    std_distance = np.std(distances)
    print(f"Group {group} - Mean: {mean_distance}, Median: {median_distance}, Std: {std_distance}")
    
    # Set threshold based on analysis
    # For example, threshold = median_distance + std_distance
    threshold = median_distance + std_distance
    # Proceed with clustering as in previous step using this threshold
